In [ ]:
# Install required libraries
!pip install pandas scikit-learn tensorflow

# Import necessary libraries
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, BackupAndRestore, EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
from google.colab import files

# Load chess game data
file_path = 'carlsen_games_moves.csv'  # Update with the actual file path
chess_data = pd.read_csv(file_path)

# Preprocessing the data
# Extract features (move_sequence) and labels (notation)
X = chess_data['move_sequence']
y = chess_data['notation']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Tokenize move sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X)
X_tokenized = tokenizer.texts_to_sequences(X)
X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_tokenized, padding='post')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

In [ ]:
# Define the model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=256),
    LSTM(256, return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Build the model explicitly to avoid the BackupAndRestore error
model.build(input_shape=(None, X_padded.shape[1]))

# Recompile the model after building
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Set up a backup and restore mechanism
backup_dir = './backup'
os.makedirs(backup_dir, exist_ok=True)
backup_and_restore = BackupAndRestore(backup_dir)

# Train the model with checkpointing and backup
history = model.fit(
    X_train,
    y_train,
    epochs=1,
    validation_data=(X_test, y_test),
    batch_size=128,
    callbacks=[backup_and_restore]
)

# Save the final model
model.save('chess_tactics_chatbot.keras')

# Automatically download the model
files.download('chess_tactics_chatbot.keras')